In [1]:
!pip install tensorflow
!pip install pillow
!pip install scipy

In [ ]:
import tensorflow as tf
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from image_utils import split_and_organize_images, create_data_set_from_images
from cnn_builder import CNNBuilder

In [ ]:
directories = dict(
    src="../data",
    output=r"c:\temp\imagenet224",
    pickle=r"c:\temp\pickle"
)

src_image_path = Path(os.path.join(os.getcwd(), directories["src"]))
output_path = Path(directories["output"])
split_and_organize_images(src_image_path, output_path, max_workers=8)

In [ ]:
training_set = create_data_set_from_images(os.path.join(directories["output"], "train"))
test_set = create_data_set_from_images(os.path.join(directories["output"], "test"))

In [ ]:
builder = CNNBuilder()
builder.add_input(kernel_size=7)
builder.add_max_pool()
builder.add_convolution(filters=192, kernel_size=3, strides=(1, 1))
builder.add_max_pool()
builder.add_dropout(0.4)
builder.add_flatten()
builder.add_output_layer(10, activation="softmax")

model = builder.build()
print(model.summary())

In [ ]:
training_result = model.fit(x=training_set, validation_data=test_set, epochs=25)

In [ ]:
if not os.path.exists(directories["pickle"]):
    os.makedirs(directories["pickle"])
    
with open(os.path.join(directories["pickle"], "training_results_control"), "wb") as file:
    pickle.dump(training_result, file)

In [ ]:
def plot_accuracy_curve(training_result):
    ACCURACY_METRIC_NAME = "accuracy"
    VAL_ACCURACY_METRIC_NAME = "val_accuracy"
    
    train_perf = training_result.history[ACCURACY_METRIC_NAME]
    validation_perf = training_result.history[VAL_ACCURACY_METRIC_NAME]
    
    plt.plot(train_perf, label=ACCURACY_METRIC_NAME)
    plt.plot(validation_perf, label=VAL_ACCURACY_METRIC_NAME)
    
    max_val_accuracy = max(validation_perf)
    max_val_accuracy_epoch = validation_perf.index(max_val_accuracy)
    
    plt.annotate(f"Max accuracy: {max_val_accuracy*100:.0f}% \n@ Epoch {max_val_accuracy_epoch}",
                 xy=(max_val_accuracy_epoch, max_val_accuracy),
                 xytext=(max_val_accuracy_epoch, max_val_accuracy + 0.05),
                 arrowprops=dict(facecolor="black", shrink=0.05))
    
    plt.xlabel("Epoch")
    plt.ylabel(ACCURACY_METRIC_NAME)
    plt.legend(loc="lower right")
    
plot_accuracy_curve(training_result)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
validation_set = create_data_set_from_images(os.path.join(directories["output"], "val"))
validation_predictions = model.predict(validation_set)